In [1]:
import pandas as pd
import numpy as np
hotel_df = pd.read_csv('D:/專案/pycon2017-optimizing-pandas-master/new_york_hotels.csv')
hotel_df

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900
...,...,...,...,...,...,...,...,...,...,...,...
1626,324259,Residence Inn Yonkers Westchester County,7 Executive Blvd,Yonkers,NY,10701,40.97275,-73.88075,3.0,269.3600,169.3600
1627,107949,Ramada Inn Yonkers,125 Tuckahoe Rd,Yonkers,NY,10710,40.95466,-73.86483,2.5,129.0000,119.0000
1628,509723,Hyatt Place New York/Yonkers,7000 Mall Walk,Yonkers,NY,10704,40.92625,-73.85438,3.0,249.3100,199.3100
1629,621870,Hampton Inn & Suites Yonkers - Westchester,555 Tuckahoe Rd,Yonkers,NY,10710,40.95375,-73.84935,2.5,189.1900,134.1800


In [2]:
def haversine(lat1,lon1,lat2,lon2):
    MILES = 3959
    lat1, lon1, lat2 , lon2 = map(np.deg2rad,[lat1,lon1,lat2,lon2])
    dlat = lat2-lat1
    dlon = lon2-lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    total_miles = MILES *c
    return total_miles

In [3]:
len(hotel_df)
print(hotel_df.loc[0,['latitude']])

latitude    42.6875
Name: 0, dtype: object


In [4]:
#crude loop
def haversine_looping(df):
    distance_list = []
    for i in range(0,len(df)):
        d = haversine(40.671, -73.985 , df.loc[i,['latitude']].astype(float), df.loc[i,['longitude']].astype(float))
        distance_list.append(d)
    return distance_list

In [5]:
%%timeit
hotel_df['distance'] = haversine_looping(hotel_df)

6.52 s ± 149 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
haversine_series = []
for index,row in hotel_df.iterrows():
        haversine_series.append(haversine(40.671, -73.985 , row['latitude'], row['longitude']))
hotel_df['distance'] = haversine_series

212 ms ± 43.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
hotel_df['distance'] = hotel_df.apply(lambda row: haversine(40.671, -73.985 , row['latitude'], row['longitude']),axis=1)

51.2 ms ± 3.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%%timeit
hotel_df['distance'] = haversine(40.671, -73.985 , hotel_df['latitude'], hotel_df['longitude'])

2.55 ms ± 355 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit
hotel_df['distance'] = haversine(40.671, -73.985 , hotel_df['latitude'].values, hotel_df['longitude'].values)

243 µs ± 54.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
